In [17]:
import sqlite3 as sql
from pathlib import Path

def initialize_database(database_file, sql_paths: list[Path]):
    # Connect to SQLite database (creates a new file if it doesn't exist)
    connection = sql.connect(database_file)

    # Create a cursor object to execute SQL commands
    cursor = connection.cursor()

    # Read and execute the SQL commands from the .sql file
    for sql_path in sql_paths:
        with sql_path.open(mode="r", encoding="utf-8") as sql_file:
            print(sql_path)
            sql_script = sql_file.read()
            cursor.executescript(sql_script)

    # Commit the changes and close the connection
    connection.commit()
    connection.close()


def query(qstr: str, conn: sql.Connection = None) -> list:
    if conn is not None:
        cur = conn.cursor()
        r = cur.execute(qstr)
        return r.fetchall()
        
    with sql.connect("pokemon.db") as conn:
        cur = conn.cursor()
        r = cur.execute(qstr.strip())
        return r.fetchall()


def list_table_schemas(database_file):
    # Connect to SQLite database
    with sql.connect(database_file) as conn:
        cursor = conn.cursor()

        # Query to retrieve table names and their schema
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        table_names = cursor.fetchall()

        # Iterate through tables and print their schema
        for table in table_names:
            table_name = table[0]
            print(f"Table: {table_name}")

            # Query to retrieve the schema of each table
            cursor.execute(f"PRAGMA table_info({table_name});")
            schema_info = cursor.fetchall()

            # Print the schema information
            for column in schema_info:
                column_name = column[1]
                column_type = column[2]
                print(f"  {column_name}: {column_type}")

            print("\n")


## Initialize DB

In [ ]:
sql_paths = list((Path.cwd() / 'sql_scripts'/ 'pokemon_db').glob("*"))
# Replace 'your_database.db' and 'your_script.sql' with your actual file names
initialize_database('pokemon.db', sql_paths[1:])

In [14]:
list_table_schemas("pokemon.db")

Table: pokemons
  id: int auto_increment
  name: TEXT
  primary_type: INT
  secondary_type: INT


Table: pokemon_trainer
  trainerID: INT
  pokelevel: INT
  hp: INT
  maxhp: INT
  attack: INT
  defense: INT
  spatk: INT
  spdef: INT
  speed: INT
  pokemon_id: INT


Table: pokemon
  id: int auto_increment
  name: TEXT
  primary_type: INT
  secondary_type: INT


Table: trainers
  trainerID: INT
  trainername: TEXT


Table: trainers_pokemon
  trainerID: INT
  pokelevel: INT
  hp: INT
  maxhp: INT
  attack: INT
  defense: INT
  spatk: INT
  spdef: INT
  speed: INT
  pokemon_id: INT


Table: types
  id: int auto_increment
  name: TEXT




## Queries

In [ ]:
q = """
SELECT trainername
FROM trainers
"""

query(q)

Names of the 5 strongest trainers

In [27]:
q = """
SELECT tr.trainername, SUM(pt.spatk) as ttl_spa, pt.*
FROM trainers as tr
JOIN pokemon_trainer as pt on pt.trainerID = tr.trainerID
GROUP BY pt.trainerID
ORDER BY ttl_spa DESC
LIMIT 20
"""

# ORDER BY (pt.hp + pt.attack + pt.defense + pt.spatk + pt.spdef + pt.speed) DESC

query(q)

[('Cooltrainer♀ Rask',
  1456,
  4146,
  100,
  446,
  446,
  361,
  241,
  231,
  171,
  241,
  104),
 ('Cooltrainer♀ Rask',
  1456,
  1320,
  100,
  446,
  446,
  361,
  241,
  231,
  171,
  241,
  104),
 ('Bird Keeper Adam', 1446, 5085, 100, 328, 328, 309, 231, 241, 241, 201, 341),
 ('Bird Keeper Adam', 1446, 2259, 100, 328, 328, 309, 231, 241, 241, 201, 341),
 ('Cooltrainer♂ Forgon',
  1426,
  4138,
  100,
  226,
  226,
  181,
  301,
  161,
  301,
  91,
  113),
 ('Cooltrainer♂ Forgon',
  1426,
  1312,
  100,
  226,
  226,
  181,
  301,
  161,
  301,
  91,
  113),
 ('Birdboy', 1416, 5082, 100, 326, 326, 211, 241, 231, 291, 211, 425),
 ('Birdboy', 1416, 2256, 100, 326, 326, 211, 241, 231, 291, 211, 425),
 ('Cool♀', 1406, 4096, 100, 266, 266, 191, 211, 241, 211, 271, 38),
 ('Cool♀', 1406, 1270, 100, 266, 266, 191, 211, 241, 211, 271, 38),
 ('Birdboy', 1394, 5083, 100, 326, 326, 211, 241, 231, 291, 211, 425),
 ('Birdboy', 1394, 2257, 100, 326, 326, 211, 241, 231, 291, 211, 425),
 ('Coo